<a href="https://colab.research.google.com/github/DhanvinS/Deep-Learning/blob/main/Neural%20Networks/An%20Analysis%20on%20CNN%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!unzip archive.zip -d flowers

Archive:  archive.zip
  inflating: flowers/flowers/bougainvillea_00002.jpg  
  inflating: flowers/flowers/bougainvillea_00003.jpg  
  inflating: flowers/flowers/bougainvillea_00004.jpg  
  inflating: flowers/flowers/bougainvillea_00005.jpg  
  inflating: flowers/flowers/bougainvillea_00006.jpg  
  inflating: flowers/flowers/bougainvillea_00007.jpg  
  inflating: flowers/flowers/bougainvillea_00008.jpg  
  inflating: flowers/flowers/bougainvillea_00009.jpg  
  inflating: flowers/flowers/bougainvillea_00010.jpg  
  inflating: flowers/flowers/bougainvillea_00012.jpg  
  inflating: flowers/flowers/bougainvillea_00013.jpg  
  inflating: flowers/flowers/bougainvillea_00014.jpg  
  inflating: flowers/flowers/bougainvillea_00015.jpg  
  inflating: flowers/flowers/bougainvillea_00016.jpg  
  inflating: flowers/flowers/bougainvillea_00017.jpg  
  inflating: flowers/flowers/bougainvillea_00018.jpg  
  inflating: flowers/flowers/bougainvillea_00019.jpg  
  inflating: flowers/flowers/bougainvillea_

In [16]:
# gonna use tensorflow
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [18]:
IMG_SIZE = (200, 200)

In [22]:
IMAGE_DIR = "flowers/flowers"

images = []
labels = []

for file in os.listdir(IMAGE_DIR):
    if file.lower().endswith(".jpg"):
        label = file.split("_")[0].lower()
        img = load_img(os.path.join(IMAGE_DIR, file), target_size=(200, 200))
        img = img_to_array(img) / 255.0

        images.append(img)
        labels.append(label)

In [23]:
X = np.array(images)
y = np.array(labels)

My dataset all classes are in same folder so gonna encode and parallelly get all types of classes we have

In [24]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded)

In [25]:
print("Classes:", encoder.classes_)
print("Number of classes:", len(encoder.classes_))

Classes: ['bougainvillea' 'daisies' 'garden' 'gardenias' 'hibiscus' 'hydrangeas'
 'lilies' 'orchids' 'peonies' 'tulip']
Number of classes: 10


In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot
)
# gonna 80/20 split my data

# TASK 1

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [28]:
model_cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:
history_cnn = model_cnn.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test)
)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 713ms/step - accuracy: 0.1008 - loss: 2.8520 - val_accuracy: 0.1905 - val_loss: 2.1414
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.2484 - loss: 2.0704 - val_accuracy: 0.3741 - val_loss: 1.7422
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.4268 - loss: 1.6615 - val_accuracy: 0.3946 - val_loss: 1.7107
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.5148 - loss: 1.4261 - val_accuracy: 0.4286 - val_loss: 1.5955
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.6035 - loss: 1.1623 - val_accuracy: 0.4762 - val_loss: 1.5482
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.6246 - loss: 1.0808 - val_accuracy: 0.5442 - val_loss: 1.4333
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.7228 - loss: 0.7758 - val_accuracy: 0.4422 - val_loss: 2.1576
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.7789 - loss: 0.5940 - val_accuracy: 0.5238 -

In [31]:
cnn_loss, cnn_acc = model_cnn.evaluate(X_test, y_test)
cnn_acc

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.5795 - loss: 3.0718


0.5782312750816345

# TASK 2

In [32]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout

In [34]:
conv_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(200,200,3)
)
conv_base.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [35]:
model_vgg = Sequential([
    conv_base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [36]:
model_vgg.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [37]:
history_vgg = model_vgg.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_test, y_test)
)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.1981 - loss: 5.2754 - val_accuracy: 0.4082 - val_loss: 2.2517
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 368ms/step - accuracy: 0.4458 - loss: 1.9662 - val_accuracy: 0.6667 - val_loss: 1.1134
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 358ms/step - accuracy: 0.7011 - loss: 0.9506 - val_accuracy: 0.7075 - val_loss: 0.9017
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 357ms/step - accuracy: 0.7796 - loss: 0.7033 - val_accuracy: 0.7347 - val_loss: 0.8134
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 422ms/step - accuracy: 0.8279 - loss: 0.5039 - val_accuracy: 0.7551 - val_loss: 0.6981
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 356ms/step - accuracy: 0.8726 - loss: 0.4020 - val_accuracy: 0.7483 - val_loss: 0.7452
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 354ms/step - accuracy: 0.9572 - loss: 0.2736 - val_accuracy: 0.7619 - val_loss: 0.6537
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 355ms/step - accuracy: 0.9386 - loss: 0.2492 - val_accuracy: 0.80

In [38]:
vgg_loss, vgg_acc = model_vgg.evaluate(X_test, y_test)
vgg_acc

5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.8180 - loss: 0.5266


0.8095238208770752

In [39]:
# doing finl comparison
print(f"Custom CNN Accuracy : {cnn_acc:.4f}")
print(f"VGG16 Accuracy      : {vgg_acc:.4f}")

Custom CNN Accuracy : 0.5782
VGG16 Accuracy      : 0.8095
